In [1]:
import logging
logging.basicConfig(filename=u"main.log", format=u'%(filename)s[LINE:%(lineno)d]# %(levelname)-8s [%(asctime)s]  %(message)s', level=logging.DEBUG)

logging.debug( u'debug' )
logging.info( u'info' )

In [2]:
import re
import time
from bs4 import BeautifulSoup
import requests
import json
#import Levenshtein as lv
import numpy as np
from gensim import corpora, models, similarities
import spacy

import pandas as pd
#for findDOI
import feedparser
import jellyfish
import copy
import os

In [3]:
from rss import rss # список рассылки
from findDOI import findDOI # функция поиска doi по названию
from ALT import altmetrics # функция поиска альтметрик (нужно понять, какие альтметрики нам нужны)
from stoplist import stoplist #стоп-лист
from rssParser import Parser # класс, который парсит рсс, в датафрейм, - предлагаю его использовать для наполненния
from findJournal import findJournal # функция поиска названия журнала по названию статьи

In [4]:
# p = Parser(rss)
# df = p.main()
#df.to_csv('example_table.csv')
df = pd.read_csv('example_table.csv')

In [6]:
df.head()

,Unnamed: 0,article_name,doi,journal,summary
0,0,Advances in thermochemical conversion of woody...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...
1,1,Fate of degraded pollutants in waste gas biofi...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...
2,2,Strain engineering for microbial production of...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...
3,3,The state-of-the-art strategies of protein eng...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...
4,4,Engineering of filamentous fungi for efficient...,NaN,NaN,<p>Publication date: July–August 2019</p><p><b...


In [5]:
# example
findJournal(df.loc[1,]["article_name"])

'Biotechnology Advances'

In [6]:
nlp = spacy.load('en', parser=False, ner=False)  

### Models (gensim)

In [16]:
import gensim 
gensim.__version__

'3.7.3'

In [7]:
new_path = '/mnt/c/Users/chuvakin/Desktop/ScientificSearch/main/models_lsi/'

In [12]:
# загружаем словарь, векторное пространство, модель Lsi (вектороное пространство)
# общий корпус натренированный на pubmed 
dictionary = corpora.Dictionary.load(new_path+"pubmed_75000docs.dict")      #Here and later - the first one is PubMed-based LSI-object, other - CUB-based LSI-objects
corpus = corpora.MmCorpus(new_path+"pubmed_5docs.mm") 
lsi = models.LsiModel.load(new_path+"pubmed_75000.lsi")

# химики куб
dictionary_CUB_chem = corpora.Dictionary.load(new_path+"chem.dict")
lsi_CUB_chem = models.LsiModel.load(new_path+"chem.lsi")
corpus_CUB_chem = corpora.MmCorpus(new_path+"chem.mm")

# онко куб
dictionary_CUB_onco = corpora.Dictionary.load(new_path+"cancer.dict")
lsi_CUB_onco = models.LsiModel.load(new_path+"cancer.lsi")
corpus_CUB_onco = corpora.MmCorpus(new_path+"cancer.mm")

# аутоимунные куб
dictionary_CUB_aiz = corpora.Dictionary.load(new_path+"aiz.dict")
lsi_CUB_aiz = models.LsiModel.load(new_path+"aiz.lsi")
corpus_CUB_aiz = corpora.MmCorpus(new_path+"aiz.mm")

# инфекции куб 
dictionary_CUB_inf = corpora.Dictionary.load(new_path+"infect.dict")
lsi_CUB_inf = models.LsiModel.load(new_path+"infect.lsi")
corpus_CUB_inf = corpora.MmCorpus(new_path+"infect.mm")

# офтальмология куб
dictionary_CUB_eye = corpora.Dictionary.load(new_path+"eye.dict")
lsi_CUB_eye = models.LsiModel.load(new_path+"eye.lsi")
corpus_CUB_eye = corpora.MmCorpus(new_path+"eye.mm")

# гететические куб
dictionary_CUB_gene = corpora.Dictionary.load(new_path+"gene.dict")
lsi_CUB_gene = models.LsiModel.load(new_path+"gene.lsi")
corpus_CUB_gene = corpora.MmCorpus(new_path+"gene.mm")

In [20]:
dictionary.token2id

{'-catenin': 0,
 'ability': 1,
 'accumulate': 2,
 'action': 3,
 'active': 4,
 'activity': 5,
 'add': 6,
 'addition': 7,
 'additionally': 8,
 'analysis': 9,
 'annexin': 10,
 'antagonist': 11,
 'antibody': 12,
 'anticancer': 13,
 'antitumor': 14,
 'apoptosis': 15,
 'apoptotic': 16,
 'apply': 17,
 'assay': 18,
 'assess': 19,
 'b': 20,
 'base': 21,
 'bax': 22,
 'bc': 23,
 'bca': 24,
 'bcl-2': 25,
 'beij': 26,
 'beijing': 27,
 'bind': 28,
 'blot': 29,
 'blotting': 30,
 'buffer': 31,
 'cadherin': 32,
 'cancer': 33,
 'cancerous': 34,
 'carcinoma': 35,
 'cck8': 36,
 'cellular': 37,
 'centrifuge': 38,
 'chamber': 39,
 'china': 40,
 'clinical': 41,
 'colonic': 42,
 'colorectal': 43,
 'commonly': 44,
 'compare': 45,
 'concentration': 46,
 'considered': 47,
 'control': 48,
 'culture': 49,
 'cyclin': 50,
 'd1': 51,
 'datum': 52,
 'death': 53,
 'decrease': 54,
 'decreased': 55,
 'demonstrate': 56,
 'density': 57,
 'detect': 58,
 'detection': 59,
 'development': 60,
 'differentiation': 61,
 'dilution

### indexes

In [11]:
index = similarities.MatrixSimilarity(lsi[corpus])          #Indexes
index_CUB_chem = similarities.MatrixSimilarity(lsi_CUB_onco[corpus_CUB_chem])
index_CUB_onco = similarities.MatrixSimilarity(lsi_CUB_onco[corpus_CUB_onco])
index_CUB_aiz = similarities.MatrixSimilarity(lsi_CUB_aiz[corpus_CUB_aiz])
index_CUB_inf = similarities.MatrixSimilarity(lsi_CUB_aiz[corpus_CUB_inf])
index_CUB_eye = similarities.MatrixSimilarity(lsi_CUB_aiz[corpus_CUB_eye])
index_CUB_gene = similarities.MatrixSimilarity(lsi_CUB_aiz[corpus_CUB_gene])

AttributeError: 'NoneType' object has no attribute 'u'

-----